In [320]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re 

In [362]:
#Combines structured and unstrucutred data using BoW CountVectorizer 

def process_data(x, y):
    # Process Words
    x_words = x['first_name'].map(str) + ' ' + x['middle_name'].map(str)  + ' ' + x['lastname'].map(str) + ' ' + x['topics'].map(str)
    y_words = y['first_name'].map(str) + ' ' + y['middle_name'].map(str)  + ' ' + y['lastname'].map(str) + ' ' + y['topics'].map(str)
    
    #Total corpus is combining x and y
    total_BOW = pd.concat([x_words, y_words], sort=False)
    
    vectorizer = TfidfVectorizer()
    total_vec = vectorizer.fit(total_BOW)
    X = total_vec.transform(x_words).todense()
    X_words = pd.DataFrame(X)

    Y = total_vec.transform(y_words).todense()
    Y_words = pd.DataFrame(Y)
    
    #Process numbers
    x_num = pd.DataFrame()
    y_num = pd.DataFrame()
    

    x_num['cities'] =  [re.findall("(\d+)",i) for i in x['cities'].map(str)]
    x_num['countries'] =  [re.findall("(\d+)",i) for i in x['countries'].map(str)]
    x_num = x_num.fillna(0, inplace=True)

    y_num['cities'] =  [re.findall("(\d+)",x) for x in y['cities'].map(str)]
    y_num['countries'] =  [re.findall("(\d+)",x) for x in y['countries'].map(str)]
    y_num = y_num.fillna(0, inplace=True)
    
    return X_words.append(x_num), Y_words.append(y_num)
   # final_Y = pd.DataFrame(Y_words.append(y_num))
   # return final_X, final_Y

In [460]:
#returns euclidean distance with weighted feature if orcid are the same

def calc_euclid(x, y, final_X, final_Y, truth_table):
    distance = abs(np.linalg.norm(final_X.iloc[x] - final_Y.iloc[y]))
    if x in truth_table.id_x.tolist() and y == truth_df.id_y[truth_df[truth_df['id_x']==x].index.values.astype(int)[0]]:
        print('weight function is working.')
        distance = distance - 10000
    return distance

In [405]:
# THis function creates truth table and includes euclid distance calculated

def create_truth_table(a, b, final_X, final_Y):
    dfa = a.drop_duplicates(subset=['orcid'])
    dfb = b.drop_duplicates(subset=['orcid'])

    new_df = pd.merge(dfa, dfb, how='inner', on='orcid')
    new_df.drop(new_df.index[0], inplace=True)
    truth_df = new_df[['id_x','id_y']]
    
    #add this code in later to check euclid distance of truth table
    #truth_df['euclid'] = [calc_euclid(truth_df.id_x.iloc[i], truth_df.id_y.iloc[i], final_X, final_Y) for i in range(len(truth_df))]
    return truth_df

In [387]:
# Predict will run all of the functions in order and write a file to output
#predict(file_a, file_b):  
authors_df = pd.read_csv('authors_1019.csv', encoding='iso-8859-1')
investigators_df = pd.read_csv('investigators_837.csv', encoding='iso-8859-1')

final_X, final_Y = process_data(authors_df,investigators_df)

truth_df = create_truth_table(authors_df, investigators_df, final_X, final_Y)
results_df = pd.DataFrame()
results_df['author'] = authors_df.id


#export_csv = results_df.to_csv (r'ucb_output.csv', index = None, header=True)


In [287]:
results_df.head()

,author
0,1
1,2
2,3
3,4
4,5


In [316]:
calc_euclid(70, 789, final_X, final_Y, truth_df)


1.4097854162518282

In [415]:
print(calc_euclid(0, 1, final_X, final_Y, truth_df))

1.39968985732117


In [ ]:
result_list = []
euclid_list = []

for i in range(len(final_X)):
    min_index = 0
    min_euclid = 1000
    for j in range(len(final_Y)):
        x = calc_euclid(i, j, final_X, final_Y, truth_df)
        if x < min_euclid:
            min_euclid = x
            min_index = j+1
    result_list.append(min_index)
    euclid_list.append(min_euclid)

weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is working.
weight function is w

In [436]:
results_df['investigator'] = result_list
results_df.head(80)

,author,investigator
0,1,461
1,2,705
2,3,571
3,4,235
4,5,361
5,6,11
6,7,12
7,8,246
8,9,387
9,10,263


In [437]:
#accuracy assesment on truth data
score = 0
incorrect_x = []
incorrect_y = []
results_inv = []

for x in range(len(truth_df)):
    index_y = 0
    index_y = truth_df.iloc[x].id_x - 1
    if truth_df.iloc[x].id_y == results_df.iloc[index_y].investigator:
        score +=1
    elif truth_df.iloc[x].id_y != results_df.iloc[index_y].investigator:
        incorrect_x.append(truth_df.iloc[x].id_x)
        incorrect_y.append(truth_df.iloc[x].id_y)
        results_inv.append(results_df.iloc[index_y].investigator)

print('Accuracy on ground truth is ' + str(score / len(truth_df)*100) + '%')

Accuracy on ground truth is 53.398058252427184%


In [438]:
incorrect_df = pd.DataFrame([incorrect_x, incorrect_y, results_inv]).transpose()
incorrect_df.columns = ['true_author','true_investigator', 'incorrect_investigator']
incorrect_df.head()

,true_author,true_investigator,incorrect_investigator
0,18,17,258
1,19,19,18
2,33,30,713
3,42,38,555
4,46,40,220


In [452]:
print(calc_euclid(6, 11, final_X, final_Y, truth_df))
print(calc_euclid(19, 19, final_X, final_Y, truth_df))

weight function is working. x = 6 y = 11
-9998.615988840156
weight function is working. x = 19 y = 19
-9998.590565579158


In [421]:
truth_df.shape

(206, 2)

In [428]:
truth_df[truth_df['id_x']==19].index.values.astype(int)[0]

4

In [430]:
truth_df.id_y[truth_df[truth_df['id_x']==19].index.values.astype(int)[0]]

19

In [459]:
result_list = []
euclid_list = []

for i in range(1,20):
    print('i =' + str(i))
    min_index = 0
    min_euclid = 1000
    for j in range(len(final_Y)):
        x = calc_euclid(i, j, final_X, final_Y, truth_df)
        print('i = ' + str(i) + 'x = ' + str(x) + ' and min_euclid = ' + str(min_euclid) + ' j = ' + str(j) + ' min_index = ' + str(min_index))
        if x < min_euclid:
            #print('min_euclid' + str(min_euclid) + ' and index is ' + str(min_index))
            min_euclid = x
            min_index = j+1
    result_list.append(min_index)
    euclid_list.append(min_euclid)

i =1
i = 1x = 1.4107196028389353 and min_euclid = 1000 j = 0
i = 1x = 1.3824274703927393 and min_euclid = 1.4107196028389353 j = 1
i = 1x = 1.3907843379098193 and min_euclid = 1.3824274703927393 j = 2
i = 1x = 1.4051097733549467 and min_euclid = 1.3824274703927393 j = 3
i = 1x = 1.2581119652737025 and min_euclid = 1.3824274703927393 j = 4
i = 1x = 1.41065197379454 and min_euclid = 1.2581119652737025 j = 5
i = 1x = 1.4006936830164995 and min_euclid = 1.2581119652737025 j = 6
i = 1x = 1.4025916972642594 and min_euclid = 1.2581119652737025 j = 7
i = 1x = 1.3971308527238833 and min_euclid = 1.2581119652737025 j = 8
i = 1x = 1.4115150304333643 and min_euclid = 1.2581119652737025 j = 9
i = 1x = 1.385484872485512 and min_euclid = 1.2581119652737025 j = 10
i = 1x = 1.4077631690889068 and min_euclid = 1.2581119652737025 j = 11
i = 1x = 1.3888190839024446 and min_euclid = 1.2581119652737025 j = 12
i = 1x = 1.4064515040257044 and min_euclid = 1.2581119652737025 j = 13
i = 1x = 1.3935483308115524 

i = 1x = 1.4097323058010707 and min_euclid = 1.2581119652737025 j = 217
i = 1x = 1.4119380119625888 and min_euclid = 1.2581119652737025 j = 218
i = 1x = 1.3817870358923967 and min_euclid = 1.2581119652737025 j = 219
i = 1x = 1.4099061483729858 and min_euclid = 1.2581119652737025 j = 220
i = 1x = 1.4040893656187692 and min_euclid = 1.2581119652737025 j = 221
i = 1x = 1.4055211326175987 and min_euclid = 1.2581119652737025 j = 222
i = 1x = 1.3723353157183174 and min_euclid = 1.2581119652737025 j = 223
i = 1x = 1.3903580775057371 and min_euclid = 1.2581119652737025 j = 224
i = 1x = 1.3636561210654639 and min_euclid = 1.2581119652737025 j = 225
i = 1x = 1.372540249055126 and min_euclid = 1.2581119652737025 j = 226
i = 1x = 1.3992804601174935 and min_euclid = 1.2581119652737025 j = 227
i = 1x = 1.4114247000544067 and min_euclid = 1.2581119652737025 j = 228
i = 1x = 1.3202014789840273 and min_euclid = 1.2581119652737025 j = 229
i = 1x = 1.3961410858705672 and min_euclid = 1.2581119652737025 j

i = 1x = 1.3966914227291611 and min_euclid = 1.2461586805943696 j = 434
i = 1x = 1.408873609678651 and min_euclid = 1.2461586805943696 j = 435
i = 1x = 1.4094810595366731 and min_euclid = 1.2461586805943696 j = 436
i = 1x = 1.4109212552378685 and min_euclid = 1.2461586805943696 j = 437
i = 1x = 1.3965743095188323 and min_euclid = 1.2461586805943696 j = 438
i = 1x = 1.4114501363356213 and min_euclid = 1.2461586805943696 j = 439
i = 1x = 1.4051097733549467 and min_euclid = 1.2461586805943696 j = 440
i = 1x = 1.4112956091540534 and min_euclid = 1.2461586805943696 j = 441
i = 1x = 1.4142135623730951 and min_euclid = 1.2461586805943696 j = 442
i = 1x = 1.4064337557818651 and min_euclid = 1.2461586805943696 j = 443
i = 1x = 1.4142135623730951 and min_euclid = 1.2461586805943696 j = 444
i = 1x = 1.3845471000385008 and min_euclid = 1.2461586805943696 j = 445
i = 1x = 1.41077226089935 and min_euclid = 1.2461586805943696 j = 446
i = 1x = 1.408525114932025 and min_euclid = 1.2461586805943696 j = 

i = 1x = 1.4073612120850731 and min_euclid = 1.2436109198504726 j = 670
i = 1x = 1.4062082447732882 and min_euclid = 1.2436109198504726 j = 671
i = 1x = 1.4001370730288711 and min_euclid = 1.2436109198504726 j = 672
i = 1x = 1.3225860763054496 and min_euclid = 1.2436109198504726 j = 673
i = 1x = 1.410350200529201 and min_euclid = 1.2436109198504726 j = 674
i = 1x = 1.4142135623730951 and min_euclid = 1.2436109198504726 j = 675
i = 1x = 1.409257463314518 and min_euclid = 1.2436109198504726 j = 676
i = 1x = 1.4112414602564782 and min_euclid = 1.2436109198504726 j = 677
i = 1x = 1.4097263917182397 and min_euclid = 1.2436109198504726 j = 678
i = 1x = 1.4043574654423407 and min_euclid = 1.2436109198504726 j = 679
i = 1x = 1.3669715585612514 and min_euclid = 1.2436109198504726 j = 680
i = 1x = 1.393172151088721 and min_euclid = 1.2436109198504726 j = 681
i = 1x = 1.3323781060415623 and min_euclid = 1.2436109198504726 j = 682
i = 1x = 1.4049335785035242 and min_euclid = 1.2436109198504726 j =

i = 2x = 1.4083806228347393 and min_euclid = 1.3992369936323392 j = 65
i = 2x = 1.4142135623730951 and min_euclid = 1.3992369936323392 j = 66
i = 2x = 1.4083146090677041 and min_euclid = 1.3992369936323392 j = 67
i = 2x = 1.4103802678368507 and min_euclid = 1.3992369936323392 j = 68
i = 2x = 1.4102162748138303 and min_euclid = 1.3992369936323392 j = 69
i = 2x = 1.4067602194629718 and min_euclid = 1.3992369936323392 j = 70
i = 2x = 1.4103110057061863 and min_euclid = 1.3992369936323392 j = 71
i = 2x = 1.4076725528765488 and min_euclid = 1.3992369936323392 j = 72
i = 2x = 1.4093300427209141 and min_euclid = 1.3992369936323392 j = 73
i = 2x = 1.4086503056707642 and min_euclid = 1.3992369936323392 j = 74
i = 2x = 1.4066544114176827 and min_euclid = 1.3992369936323392 j = 75
i = 2x = 1.4142135623730951 and min_euclid = 1.3992369936323392 j = 76
i = 2x = 1.4060347306232306 and min_euclid = 1.3992369936323392 j = 77
i = 2x = 1.4142135623730951 and min_euclid = 1.3992369936323392 j = 78
i = 2x

...

i = 18x = 1.3268049814238785 and min_euclid = -9998.596141098913 j = 715
i = 18x = 1.4028755796790462 and min_euclid = -9998.596141098913 j = 716
i = 18x = 1.3967790665104862 and min_euclid = -9998.596141098913 j = 717
i = 18x = 1.3670223834408706 and min_euclid = -9998.596141098913 j = 718
i = 18x = 1.3592722977552703 and min_euclid = -9998.596141098913 j = 719
i = 18x = 1.4019895215285199 and min_euclid = -9998.596141098913 j = 720
i = 18x = 1.306133019796892 and min_euclid = -9998.596141098913 j = 721
i = 18x = 1.3913148001017601 and min_euclid = -9998.596141098913 j = 722
i = 18x = 1.400649687526794 and min_euclid = -9998.596141098913 j = 723
i = 18x = 1.4034320496657875 and min_euclid = -9998.596141098913 j = 724
i = 18x = 1.411650214342064 and min_euclid = -9998.596141098913 j = 725
i = 18x = 1.4073812085309902 and min_euclid = -9998.596141098913 j = 726
i = 18x = 1.3849900095500407 and min_euclid = -9998.596141098913 j = 727
i = 18x = 1.4094231153262076 and min_euclid = -9998.59

i = 19x = 1.4058472169336815 and min_euclid = 1.4103994462897527 j = 1
i = 19x = 1.4098401121536415 and min_euclid = 1.4058472169336815 j = 2
i = 19x = 1.404273758886693 and min_euclid = 1.4058472169336815 j = 3
i = 19x = 1.3848178288226507 and min_euclid = 1.404273758886693 j = 4
i = 19x = 1.4103256117461813 and min_euclid = 1.3848178288226507 j = 5
i = 19x = 1.404944445218829 and min_euclid = 1.3848178288226507 j = 6
i = 19x = 1.406244009313371 and min_euclid = 1.3848178288226507 j = 7
i = 19x = 1.4035333754918804 and min_euclid = 1.3848178288226507 j = 8
i = 19x = 1.4112678362247588 and min_euclid = 1.3848178288226507 j = 9
i = 19x = 1.4088591515222828 and min_euclid = 1.3848178288226507 j = 10
i = 19x = 1.4071714316206385 and min_euclid = 1.3848178288226507 j = 11
i = 19x = 1.4094759754643562 and min_euclid = 1.3848178288226507 j = 12
i = 19x = 1.4088885242187186 and min_euclid = 1.3848178288226507 j = 13
i = 19x = 1.4000570727009924 and min_euclid = 1.3848178288226507 j = 14
i = 1

i = 19x = 1.4027337164004041 and min_euclid = -9998.590565579158 j = 227
i = 19x = 1.4111692223585206 and min_euclid = -9998.590565579158 j = 228
i = 19x = 1.402382098500381 and min_euclid = -9998.590565579158 j = 229
i = 19x = 1.4018295785094081 and min_euclid = -9998.590565579158 j = 230
i = 19x = 1.4100181011784165 and min_euclid = -9998.590565579158 j = 231
i = 19x = 1.4097481568190713 and min_euclid = -9998.590565579158 j = 232
i = 19x = 1.4114804892061685 and min_euclid = -9998.590565579158 j = 233
i = 19x = 1.405415916935611 and min_euclid = -9998.590565579158 j = 234
i = 19x = 1.3980516753186407 and min_euclid = -9998.590565579158 j = 235
i = 19x = 1.4088415905686504 and min_euclid = -9998.590565579158 j = 236
i = 19x = 1.4052817773556987 and min_euclid = -9998.590565579158 j = 237
i = 19x = 1.4113970828960107 and min_euclid = -9998.590565579158 j = 238
i = 19x = 1.4112229555401596 and min_euclid = -9998.590565579158 j = 239
i = 19x = 1.4003481020100468 and min_euclid = -9998.5

i = 19x = 1.3801028905632091 and min_euclid = -9998.590565579158 j = 447
i = 19x = 1.4096067215181292 and min_euclid = -9998.590565579158 j = 448
i = 19x = 1.4026212211059388 and min_euclid = -9998.590565579158 j = 449
i = 19x = 1.4072481728840323 and min_euclid = -9998.590565579158 j = 450
i = 19x = 1.4108893225668369 and min_euclid = -9998.590565579158 j = 451
i = 19x = 1.4092657666858477 and min_euclid = -9998.590565579158 j = 452
i = 19x = 1.4115933476750868 and min_euclid = -9998.590565579158 j = 453
i = 19x = 1.3999530384065246 and min_euclid = -9998.590565579158 j = 454
i = 19x = 1.4111614604016072 and min_euclid = -9998.590565579158 j = 455
i = 19x = 1.4042219162221115 and min_euclid = -9998.590565579158 j = 456
i = 19x = 1.409062845432391 and min_euclid = -9998.590565579158 j = 457
i = 19x = 1.4105388912123704 and min_euclid = -9998.590565579158 j = 458
i = 19x = 1.4098812798767277 and min_euclid = -9998.590565579158 j = 459
i = 19x = 1.414213562373095 and min_euclid = -9998.5

i = 19x = 1.4094344208433036 and min_euclid = -9998.590565579158 j = 661
i = 19x = 1.4086070434611055 and min_euclid = -9998.590565579158 j = 662
i = 19x = 1.414213562373095 and min_euclid = -9998.590565579158 j = 663
i = 19x = 1.4096482253169624 and min_euclid = -9998.590565579158 j = 664
i = 19x = 1.4106031343426286 and min_euclid = -9998.590565579158 j = 665
i = 19x = 1.3795173558737823 and min_euclid = -9998.590565579158 j = 666
i = 19x = 1.408682168912737 and min_euclid = -9998.590565579158 j = 667
i = 19x = 1.408137321519565 and min_euclid = -9998.590565579158 j = 668
i = 19x = 1.410201647157084 and min_euclid = -9998.590565579158 j = 669
i = 19x = 1.406732502155968 and min_euclid = -9998.590565579158 j = 670
i = 19x = 1.405473419241924 and min_euclid = -9998.590565579158 j = 671
i = 19x = 1.4044883291477126 and min_euclid = -9998.590565579158 j = 672
i = 19x = 1.4083007506448388 and min_euclid = -9998.590565579158 j = 673
i = 19x = 1.4099961444294138 and min_euclid = -9998.59056

In [446]:
truth_df.head(20)

,id_x,id_y
1,6,11
2,13,14
3,18,17
4,19,19
5,33,30
6,40,37
7,42,38
8,46,40
9,47,44
10,48,45


In [402]:
def calc_euclid_test(x, y, final_X, final_Y, truth_table):
    distance = abs(np.linalg.norm(final_X.iloc[x] - final_Y.iloc[y]))
#     print(x)
#     print(truth_table.id_x)
    if x in truth_table.id_x.tolist() and y in truth_table.id_y.tolist():
        print('weight function is working')
        distance = distance - 10000
    return distance

In [54]:
predict('authors_1019.csv','investigators_837.csv')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
